# Datasets Analysis Tables

In [1]:
import numpy as np
import pandas as pd
import os

from configs.config import DATASETS
import openml

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer

In [ ]:
source = "openml_ctr23"
table_data = []

datasets_to_run = DATASETS.get(source, {})


for dataset_key, dataset_info_dict in datasets_to_run.items():
    dataset_name = dataset_info_dict.get('name', dataset_key)

    # https://docs.openml.org/intro/
    task = openml.tasks.get_task(int(dataset_key))
    dataset = task.get_dataset()
    X, y, categorical_indicator, attribute_names = dataset.get_data(target=task.target_name)  
    train_indices, test_indices = task.get_train_test_split_indices(fold=0)
    
    train_size = len(train_indices)
    test_size = len(test_indices)

    num_input_features = X.shape[1]


    if y.ndim == 1:
        num_output_features = 1
    else:
        num_output_features = y.shape[1]

    num_categorical_features = sum(categorical_indicator)



    missing_values_per_feature = X.isnull().sum()
    total_missing_cells_in_X = missing_values_per_feature.sum()
    num_input_features_with_missing = (missing_values_per_feature > 0).sum()

    total_cells_in_X = X.size
    percent_missing_overall = (total_missing_cells_in_X / total_cells_in_X) * 100 if total_cells_in_X > 0 else 0

    table_data.append({
        "ID": int(dataset_key),
        "Name": dataset_name,
        "Train Size": train_size,
        "Test Size": test_size,
        "Input Features": num_input_features,
        "Output Features": num_output_features,
        "Categorical Input Features": num_categorical_features,
        "Input Features with Missing Values": num_input_features_with_missing,
        "% Missing Values": f"{percent_missing_overall:.2f}%"
    })


df_openml = pd.DataFrame(table_data)
#df_openml.to_excel("openml_benchmark.xlsx", index=False)
print(df_openml.to_string())
    

        ID                           Name  Train Size  Test Size  Input Features  Output Features  Categorical Input Features  Input Features with Missing Values % Missing Values
0   361251                 grid_stability        9000       1000              12                1                           0                                   0            0.00%
1   361252              video_transcoding       61905       6879              18                1                           2                                   0            0.00%
2   361253                    wave_energy       64800       7200              48                1                           0                                   0            0.00%
3   361254                         sarcos       44039       4894              21                1                           0                                   0            0.00%
4   361255             california_housing       18576       2064               8                1        

In [ ]:
def load_uci_data_segment(filepath_data,
                          filepath_index_columns,
                          filepath_index_rows,
                          data_delimiter=None,
                          index_columns_delimiter=None,
                          index_rows_delimiter=None):
    """
    Loads a segment of UCI data based on data file and index files for rows and columns.
    Mimics the behavior of the provided UCIDataSet._load method.
    """
    # Load the entire data matrix
    data_full = np.loadtxt(filepath_data, delimiter=data_delimiter)
    df_full = pd.DataFrame(data_full)

    # Load column indices and reshape to be 1D
    index_columns = np.loadtxt(filepath_index_columns, dtype=np.int32, delimiter=index_columns_delimiter)
    index_columns = index_columns.reshape(-1)

    # Load row indices and reshape to be 1D
    index_rows = np.loadtxt(filepath_index_rows, dtype=np.int32, delimiter=index_rows_delimiter)
    index_rows = index_rows.reshape(-1)

    # Select the specified rows and columns
    return df_full.iloc[index_rows, index_columns]

def get_uci_scalers():
    """
    Returns the feature and target scalers as defined in ResFlowDataModule.
    """

    feature_scaler = Pipeline(
            [("quantile", QuantileTransformer(output_distribution="normal")),
             ("standarize", StandardScaler()),])

    target_scaler = MinMaxScaler(feature_range=(-1, 1))
    return feature_scaler, target_scaler

def fit_and_transform_data(X_train_raw: pd.DataFrame,
                             y_train_raw: pd.DataFrame,
                             X_test_raw: pd.DataFrame,
                             y_test_raw: pd.DataFrame):
    """
    Fits the scalers on training data and transforms train and test sets.
    Returns processed X_train, y_train, X_test, y_test as NumPy arrays,
    and the fitted scalers.
    """
    feature_scaler, target_scaler = get_uci_scalers()

    # Prepare data for scikit-learn (NumPy arrays)
    X_train_np = X_train_raw.to_numpy()
    y_train_np = y_train_raw.to_numpy()
    X_test_np = X_test_raw.to_numpy()
    y_test_np = y_test_raw.to_numpy()

    # Reshape y if it's 1D for scaler compatibility
    if y_train_np.ndim == 1:
        y_train_np = y_train_np.reshape(-1, 1)
    if y_test_np.ndim == 1:
        y_test_np = y_test_np.reshape(-1, 1)

    # Fit scalers on training data
    feature_scaler.fit(X_train_np)
    target_scaler.fit(y_train_np)

    # Transform data
    X_train_processed = feature_scaler.transform(X_train_np)
    y_train_processed = target_scaler.transform(y_train_np)
    X_test_processed = feature_scaler.transform(X_test_np)
    y_test_processed = target_scaler.transform(y_test_np)

    return (X_train_processed, y_train_processed,
            X_test_processed, y_test_processed,
            feature_scaler, target_scaler)

In [13]:
source = "uci"
table_data = []

datasets_to_run = DATASETS.get(source, {})
fold = 0

for dataset_key, dataset_info_dict in datasets_to_run.items():

    dataset_name = dataset_info_dict.get('name', dataset_key)

    current_dataset_path = os.path.join("downloaded_datasets/UCI", dataset_key)

    # Define file paths
    fp_data = os.path.join(current_dataset_path, "data.txt")
    fp_index_features = os.path.join(current_dataset_path, "index_features.txt")
    fp_index_target = os.path.join(current_dataset_path, "index_target.txt")
    fp_index_train_rows = os.path.join(current_dataset_path, f"index_train_{fold}.txt")
    fp_index_test_rows = os.path.join(current_dataset_path, f"index_test_{fold}.txt")

    required_files_info = {
        "Data File": fp_data, "Feature Index": fp_index_features,
        "Target Index": fp_index_target, "Train Row Index": fp_index_train_rows,
        "Test Row Index": fp_index_test_rows
    }

    all_files_present = True
    for name, path in required_files_info.items():
        if not os.path.exists(path):
            print(f"  ERROR: {name} not found at {path}")
            all_files_present = False
            break

    # Load raw data segments (no try-except here as per previous pattern)
    x_train_raw = load_uci_data_segment(fp_data, fp_index_features, fp_index_train_rows)
    y_train_raw = load_uci_data_segment(fp_data, fp_index_target, fp_index_train_rows)
    x_test_raw = load_uci_data_segment(fp_data, fp_index_features, fp_index_test_rows)
    y_test_raw = load_uci_data_segment(fp_data, fp_index_target, fp_index_test_rows)

    # Get characteristics from RAW loaded data
    train_size = x_train_raw.shape[0]
    test_size = x_test_raw.shape[0]
    num_input_features = x_train_raw.shape[1]

    if y_train_raw.ndim == 1:
        num_output_features = 1
    else:
        num_output_features = y_train_raw.shape[1]

    num_categorical = x_train_raw.select_dtypes(include=['object', 'category']).shape[1]



    # We combine X_train and X_test for this fold to get a view of missingness in the features
    x_combined_fold_raw = pd.concat([x_train_raw, x_test_raw], axis=0, ignore_index=True)

    missing_per_feature_combined = x_combined_fold_raw.isnull().sum()
    num_features_with_any_missing = (missing_per_feature_combined > 0).sum()
    total_missing_values_combined = missing_per_feature_combined.sum()

    total_cells_combined = x_combined_fold_raw.size # Total number of cells in the combined X data
    percent_missing_combined = (total_missing_values_combined / total_cells_combined) * 100 if total_cells_combined > 0 else 0

    table_data.append({
        "ID": dataset_key,
        "Name": dataset_name,
        "Train Size": train_size,
        "Test Size": test_size,
        "Input Features": num_input_features,
        "Output Features": num_output_features,
        "Categorical Input Features": num_categorical,
        "Input Features w/ Missing": num_features_with_any_missing,
        "% Missing": f"{percent_missing_combined:.2f}%"
    })


df_uci = pd.DataFrame(table_data)


print(df_uci.to_string())
#df_uci.to_excel("uci_benchmark.xlsx", index=False)
print("\nPreprocessing steps identified from ResFlowDataModule (applied after loading):")
print("  Features: 1. QuantileTransformer(output_distribution='normal') -> 2. StandardScaler()")
print("  Target:   1. MinMaxScaler(feature_range=(-1, 1))")

                           ID                                                      Name  Train Size  Test Size  Input Features  Output Features  Categorical Input Features  Input Features w/ Missing % Missing
0                    concrete                             Concrete Compressive Strength         927        103               8                1                           0                          0     0.00%
1                      energy                                         Energy Efficiency         691         77               8                1                           0                          0     0.00%
2                      kin8nm                                            Kinematics 8nm        7373        819               8                1                           0                          0     0.00%
3      naval-propulsion-plant    Condition Based Maintenance of Naval Propulsion Plants       10741       1193              16                1                     

# Final Table

In [17]:
print("-" * 50, "UCI", "-" * 50)
print(df_uci.to_string())

print("\n\n","-" * 50, "OpenML CTR 23", "-" * 50)
print(df_openml.to_string())

-------------------------------------------------- UCI --------------------------------------------------
                           ID                                                      Name  Train Size  Test Size  Input Features  Output Features  Categorical Input Features  Input Features w/ Missing % Missing
0                    concrete                             Concrete Compressive Strength         927        103               8                1                           0                          0     0.00%
1                      energy                                         Energy Efficiency         691         77               8                1                           0                          0     0.00%
2                      kin8nm                                            Kinematics 8nm        7373        819               8                1                           0                          0     0.00%
3      naval-propulsion-plant    Condition Based Maintenan

# OpenML-CTR23 XGBoost results from paper

In [2]:
ordered_datasets_data = [
    # From "openml_ctr23"
    {"id": "361251", "name": "grid_stability",                 "xgboost_rmse": 0.744, "power": -2},
    {"id": "361252", "name": "video_transcoding",              "xgboost_rmse": 0.078, "power": 1},
    {"id": "361253", "name": "wave_energy",                    "xgboost_rmse": 0.497, "power": 4},
    {"id": "361254", "name": "sarcos",                         "xgboost_rmse": 0.214, "power": 1},
    {"id": "361255", "name": "california_housing",             "xgboost_rmse": 4.464, "power": 4},
    {"id": "361256", "name": "cpu_activity",                   "xgboost_rmse": 2.190, "power": 0},
    {"id": "361257", "name": "diamonds",                       "xgboost_rmse": 0.521, "power": 3},
    {"id": "361258", "name": "kin8nm",                         "xgboost_rmse": 1.092, "power": -1},
    {"id": "361259", "name": "pumadyn32nh",                    "xgboost_rmse": 2.176, "power": -2},
    {"id": "361260", "name": "miami_housing",                  "xgboost_rmse": 0.815, "power": 5},
    {"id": "361261", "name": "cps88wages",                     "xgboost_rmse": 3.800, "power": 2},
    {"id": "361264", "name": "socmob",                         "xgboost_rmse": 1.246, "power": 1},
    {"id": "361266", "name": "kings_county",                   "xgboost_rmse": 1.144, "power": 5},
    {"id": "361267", "name": "brazilian_houses",               "xgboost_rmse": 0.446, "power": 4},
    {"id": "361268", "name": "fps_benchmark",                  "xgboost_rmse": 0.051, "power": 1},
    {"id": "361269", "name": "health_insurance",               "xgboost_rmse": 1.439, "power": 1},
    {"id": "361272", "name": "fifa",                           "xgboost_rmse": 0.893, "power": 4},
    {"id": "361234", "name": "abalone",                        "xgboost_rmse": 2.118, "power": 0},
    {"id": "361235", "name": "airfoil_self_noise",             "xgboost_rmse": 1.170, "power": 0},
    {"id": "361236", "name": "auction_verification",           "xgboost_rmse": 0.394, "power": 3},
    {"id": "361237", "name": "concrete_compressive_strength",  "xgboost_rmse": 0.371, "power": 1},
    {"id": "361241", "name": "physiochemical_protein",         "xgboost_rmse": 3.326, "power": 0},
    {"id": "361242", "name": "superconductivity",              "xgboost_rmse": 0.901, "power": 1},
    {"id": "361243", "name": "geographical_origin_of_music",   "xgboost_rmse": 1.519, "power": 1},
    {"id": "361244", "name": "solar_flare",                    "xgboost_rmse": 7.627, "power": -1},
    {"id": "361247", "name": "naval_propulsion_plant",         "xgboost_rmse": 0.078, "power": -2},
    {"id": "361249", "name": "white_wine",                     "xgboost_rmse": 5.693, "power": -1},
    {"id": "361250", "name": "red_wine",                       "xgboost_rmse": 5.473, "power": -1},
    {"id": "361616", "name": "Moneyball",                      "xgboost_rmse": 2.218, "power": 1},
    {"id": "361617", "name": "energy_efficiency",              "xgboost_rmse": 0.280, "power": 0},
    {"id": "361618", "name": "forest_fires",                   "xgboost_rmse": 4.830, "power": 1},
    {"id": "361619", "name": "student_performance_por",        "xgboost_rmse": 2.675, "power": 0},
    {"id": "361621", "name": "QSAR_fish_toxicity",             "xgboost_rmse": 0.864, "power": 0},
    {"id": "361622", "name": "cars",                           "xgboost_rmse": 2.111, "power": 3},
    {"id": "361623", "name": "space_ga",                       "xgboost_rmse": 1.049, "power": -1},
]


def to_superscript(s_val):
    superscript_map = {
        "0": "⁰", "1": "¹", "2": "²", "3": "³", "4": "⁴",
        "5": "⁵", "6": "⁶", "7": "⁷", "8": "⁸", "9": "⁹",
        "-": "⁻"
    }
    return "".join(superscript_map.get(char, char) for char in str(s_val))

def format_power_display(power_value):
    return f"×10{to_superscript(str(power_value))}"

output_table_data = []

for dataset in ordered_datasets_data:
    dataset_id = dataset["id"]
    dataset_name = dataset["name"]
    xgboost_rmse = dataset["xgboost_rmse"]
    power_integer = dataset["power"]

    real_rmse = xgboost_rmse * (10 ** power_integer)

    output_table_data.append({
        "id": dataset_id,
        "name": dataset_name,
        "original_rmse": xgboost_rmse,
        "power_integer": power_integer,
        "real_rmse": real_rmse
    })


# --- Print the real table (name(id) and Real RMSE only) ---
print("--- Real RMSE Summary Table ---")
header_name_id_simple = "name(id)"
header_real_rmse_simple = "Real RMSE"

print(f"{header_name_id_simple:<40} {header_real_rmse_simple:>18}")
print("-" * (40 + 1 + 18)) # Total 60 characters

for res in output_table_data:
    print(f"{res['name'] + '(' + res['id'] + ')':<40} {res['real_rmse']:>18.4g}")

# --- Print the (detailed) table ---
print("\n\n--- Detailed RMSE Table ---")
header_name_id = "name(id)"
header_orig_rmse = "Original RMSE"
header_power = "Power"
header_real_rmse = "Real RMSE"

print(f"{header_name_id:<40} {header_orig_rmse:>15} {header_power:>10} {header_real_rmse:>18}")
print("-" * (40 + 1 + 15 + 1 + 10 + 1 + 18)) # Total 86 characters

for res in output_table_data:
    power_display_string = format_power_display(res['power_integer'])
    print(f"{res['name'] + '(' + res['id'] + ')':<40} {res['original_rmse']:>15.3f} {power_display_string:>10} {res['real_rmse']:>18.4g}")

print("\n" * 2) # Add some space before the next table

--- Real RMSE Summary Table ---
name(id)                                          Real RMSE
-----------------------------------------------------------
grid_stability(361251)                              0.00744
video_transcoding(361252)                              0.78
wave_energy(361253)                                    4970
sarcos(361254)                                         2.14
california_housing(361255)                        4.464e+04
cpu_activity(361256)                                   2.19
diamonds(361257)                                        521
kin8nm(361258)                                       0.1092
pumadyn32nh(361259)                                 0.02176
miami_housing(361260)                              8.15e+04
cps88wages(361261)                                      380
socmob(361264)                                        12.46
kings_county(361266)                              1.144e+05
brazilian_houses(361267)                               4460
fps_benc